In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [2]:
# This is the overall turnover summary data
Turnover_data = pd.read_csv('Q2_2021 Turnover Data.csv')
# These are the individual turnover details
Individual_Turnover = pd.read_csv('Q2_2021 Individual Turnover.csv')

In [3]:
correct_FTEs = []
for FTE in Individual_Turnover.xTurnoverFteFID:
    if FTE == 1.0:
        correct_FTEs.append("1.00")
    elif FTE == 0.94:
        correct_FTEs.append(".90")
    elif FTE == 0.9:
        correct_FTEs.append(".90")
    elif FTE == 0.8:
        correct_FTEs.append(".80")
    elif FTE == 0.78:
        correct_FTEs.append(".80")
    elif FTE == 0.75:
        correct_FTEs.append(".75")
    elif FTE == 0.65:
        correct_FTEs.append(".65")
    elif FTE == 0.6:
        correct_FTEs.append(".60")
    elif FTE == 0.5:
        correct_FTEs.append(".50")
    elif FTE == 0.1:
        correct_FTEs.append(".10")

In [4]:
Individual_Turnover.xTurnoverFteFID = correct_FTEs

In [5]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_RN_Turnover_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    
    # The first step takes only the data for this selected hospital
    Hosp_data = Turnover_data[Turnover_data.Hospital == Selected_Hosp]
    # Now this is a List of this Hospital's Units
    Hosp_Units = list(set(Hosp_data.UnitName))

    turnover_data_list = ['RnBudgetedFtes','RnActualFtes','RnFullTime','RnPartTime','RnNumberSeparations',
                          'AprnBudgetedFtes','AprnActualFtes','AprnFullTime','AprnPartTime','AprnNumberSeparations',
                          'LpnBudgetedFtes','LpnActualFtes','LpnFullTime','LpnPartTime','LpnNumberSeparations',
                          'UapBudgetedFtes','UapActualFtes','UapFullTime','UapPartTime','UapNumberSeparations']

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is the xml's entire UnitYrQtr node
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

   # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter

    # This loops through each separate UnitName from the Template XML file and replaces it the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with the months from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            # Now take each xml month's "Turnover Summary" section
            turnover_nodes = month.getElementsByTagName("TurnoverSummary")
            for turnover_node in turnover_nodes:

                # and replace each listed metric with accurate data from Hosp_data for that unit's month. 
                for data in turnover_data_list:
                    turnover_node.getElementsByTagName(data)[0].childNodes[0].nodeValue = new_data[data].iloc[idx2]

    # This is a temporary xml file that I will now use to add the individual separation data.
    tree.writexml(open('Temp_{}.xml'.format(Selected_Hosp), 'w'))

In [6]:
# The variables needed for this function are the: Hospital abbreviation, Hospital Temp XML file,
# Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_RN_Individual_XML(Selected_Hosp,Temp_XML,Calendar_Year,Calendar_Quarter):
    
    # This Hospital's last Quarter's individual separations
    Hosp_individual = Individual_Turnover[Individual_Turnover.Hospital == Selected_Hosp]
    
    # This section is to insert the individual separations. 
    individual_turnovers = [ 'xTurnoverStaffCategoryFID','xTurnoverFteFID','xTurnoverTenureYearsFIDUnit',
                          'xTurnoverTenureYearsFIDHospital','xTurnoverSeparationTypeFID','xTurnoverPrimaryReasonFID']

    # This is the temporary xml file that has already had the "Turnover Summary" section filled
    tree = xml.dom.minidom.parse(Temp_XML)
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # This loops through each listed unit in our xml file
    for idx, unit in enumerate(unit_nodes):
        each_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # Then we loop through each listed month
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            each_month = month.getElementsByTagName('Month')[0].childNodes[0].nodeValue

            # Now at the Turnover Summary for each month
            turnover_nodes = month.getElementsByTagName("TurnoverSummary")
            for turnover_node in turnover_nodes:

                # I first check to find if any separations were listed
                RN_separations = turnover_node.getElementsByTagName('RnNumberSeparations')[0].childNodes[0].nodeValue
                APRN_separations = turnover_node.getElementsByTagName('AprnNumberSeparations')[0].childNodes[0].nodeValue
                LPN_separations = turnover_node.getElementsByTagName('LpnNumberSeparations')[0].childNodes[0].nodeValue
                UAP_separations = turnover_node.getElementsByTagName('UapNumberSeparations')[0].childNodes[0].nodeValue
                Separations = [int(RN_separations), int(APRN_separations), int(LPN_separations), int(UAP_separations)]

                Total_Separations = sum(Separations)

                # If a separation was listed for that month and that unit
                if Total_Separations > 0:

                    # Then I grab this unit's data from Hosp_individual separations
                    new_data = Hosp_individual[Hosp_individual.UnitName == each_unit]
                    # Lastly, I select only that unit's separations that occured in this specific month
                    each_turnover = new_data[new_data.Month == int(each_month)]

                    # Now I can take each listed separation
                    for i in range(len(each_turnover)):
                        # and create a "Turnover Separation" node for each individual
                        separation = tree.createElement("TurnoverSeparation")
                        for data in individual_turnovers:
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)
                            # and I add each one to that individual's "Turnover Separation" node
                            data_name.appendChild(tree.createTextNode(str(each_turnover[data].iloc[i])))

                            separation.appendChild(data_name)

                        # Finally, I can add these individual separations to the turnover summary section.
                        turnover_node.getElementsByTagName("TurnoverSeparations")[0].appendChild(separation)


    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Turnover.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [7]:
NDNQI_RN_Turnover_XML('EUOSH','2020 EUOSH Turnover Template.xml','SBI556',2021,2)

In [8]:
NDNQI_RN_Individual_XML('EUOSH','Temp_EUOSH.xml',2021,2)

In [9]:
# This Hospital's Turnover Summary Data
EUOSH_data = Turnover_data[Turnover_data.Hospital == 'EUOSH']
# This Hospital's last Quarter's individual separations
EUOSH_individual = Individual_Turnover[Individual_Turnover.Hospital == 'EUOSH']

In [10]:
EUOSH_data.head()

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
3,EUOSH,OSH POHA/PAT,4,2.79,3.0,3,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.51,1.0,1,0,0
16,EUOSH,OSH POHA/PAT,6,2.79,3.0,3,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.51,1.0,1,0,0
50,EUOSH,OSH POHA/PAT,5,2.79,3.0,3,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.51,1.0,1,0,0
65,EUOSH,OSH ICU,5,6.58,6.0,6,1,0,0.0,0.0,...,0.0,0.0,0,0,0,0.04,0.0,0,0,0
92,EUOSH,OSH PACU,6,9.44,9.9,8,4,2,0.0,0.0,...,0.0,0.0,0,0,0,1.55,1.6,1,1,0


In [11]:
EUOSH_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
0,EUOSH,OSH ICU,4,Less than 1 year,4 years,RN,.90,Left hospital,Family obligations
1,EUOSH,OSH OR,4,1 year,3 years,RN,1.00,"Stayed on unit, left permanent direct care pos...",Unknown reason
2,EUOSH,OSH OR,4,2 years,5 years,UAP,.90,"Stayed on unit, left permanent direct care pos...",Unknown reason
191,EUOSH,OSH2,6,13 years,19 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
192,EUOSH,OSH PACU,6,13 years,14 years,RN,1.00,"Stayed in hospital, left unit",Unknown reason


## EUH

In [12]:
NDNQI_RN_Turnover_XML('EUH','2020 EUH Turnover Template.xml','JXE432',2021,2)

In [13]:
NDNQI_RN_Individual_XML('EUH','Temp_EUH.xml',2021,2)

In [14]:
# This Hospital's Turnover Summary Data
EUH_data = Turnover_data[Turnover_data.Hospital == 'EUH']
# This Hospital's last Quarter's individual separations
EUH_individual = Individual_Turnover[Individual_Turnover.Hospital == 'EUH']

In [15]:
EUH_data.head()

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
0,EUH,5G Medical,6,17.50,11.9,13,0,1,0.00,0.0,...,0.0,0.0,0,0,0,7.18,8.1,9,0,1
1,EUH,EUH ED,4,67.62,67.0,72,3,3,0.14,0.0,...,0.0,0.0,0,0,0,17.83,14.5,12,6,1
4,EUH,6E CV,5,29.87,31.3,34,1,1,1.03,1.0,...,0.0,0.0,0,0,0,13.11,13.6,15,0,0
7,EUH,7G (Medical),5,38.95,28.9,32,0,2,0.93,1.0,...,0.0,0.0,0,0,0,16.48,15.4,17,0,4
10,EUH,EUH ED,5,67.62,62.5,67,3,6,0.14,0.0,...,0.0,0.0,0,0,0,17.83,14.5,12,6,1


In [16]:
EUH_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
95,EUH,6G Medical,4,2 years,2 years,RN,.90,"Stayed on unit, left permanent direct care pos...",Unknown reason
96,EUH,5G Medical,4,Less than 1 year,Less than 1 year,RN,.90,"Stayed on unit, left permanent direct care pos...",Unknown reason
97,EUH,5G Medical,4,3 years,17 years,UAP,.90,Left hospital,Other reason not listed above
98,EUH,5D Med,4,4 years,Less than 1 year,RN,.90,Left hospital,Join travel agency
99,EUH,5D Med,4,4 years,4 years,RN,.90,Left hospital,Join travel agency


## ESJH

In [17]:
NDNQI_RN_Turnover_XML('ESJH','2020 ESJH Turnover Template.xml','XAR317',2021,2)

In [18]:
NDNQI_RN_Individual_XML('ESJH','Temp_ESJH.xml',2021,2)

In [19]:
# This Hospital's Turnover Summary Data
ESJH_data = Turnover_data[Turnover_data.Hospital == 'ESJH']
# This Hospital's last Quarter's individual separations
ESJH_individual = Individual_Turnover[Individual_Turnover.Hospital == 'ESJH']

In [20]:
ESJH_data.head()

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
5,ESJH,5 West Internal Medicine,5,11.32,17.8,19,1,0,0.32,0.0,...,0.0,0.0,0,0,0,5.52,5.4,6,0,1
6,ESJH,5 West Internal Medicine,6,11.32,17.8,19,1,0,0.32,0.0,...,0.0,0.0,0,0,0,5.52,5.4,6,0,0
15,ESJH,4 East Telemetry (Neurovascular),5,22.27,23.1,25,1,2,0.48,0.0,...,0.0,0.0,0,0,0,8.90,7.3,8,0,0
18,ESJH,2 South CV Step-down,5,22.82,6.3,7,0,2,0.34,0.0,...,0.0,0.0,0,0,0,6.25,5.4,4,3,0
22,ESJH,2 Southwest ICU,4,40.38,44.7,39,16,2,0.93,1.0,...,0.0,0.0,0,0,0,2.92,2.8,3,0,0


In [21]:
ESJH_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
160,ESJH,6 East Telemetry (Internal medicine),4,2 years,2 years,UAP,.90,"Stayed in hospital, left unit",Unknown reason
161,ESJH,ESJH 7 West Complex Med-Surg,4,5 years,5 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
162,ESJH,3 East Telemetry (CV Step- down),4,2 years,4 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
163,ESJH,6 West Telemetry (Pulmonary-thoracic),4,3 years,3 years,RN,.90,Left hospital,Obtained position with a more desirable commute
164,ESJH,6 West Telemetry (Pulmonary-thoracic),4,3 years,3 years,UAP,.90,"Stayed in hospital, left unit",Unknown reason


## EUHM

In [22]:
NDNQI_RN_Turnover_XML('EUHM','2020 EUHM Turnover Template.xml','PVF155',2021,2)

In [23]:
NDNQI_RN_Individual_XML('EUHM','Temp_EUHM.xml',2021,2)

In [24]:
# This Hospital's Turnover Summary Data
EUHM_data = Turnover_data[Turnover_data.Hospital == 'EUHM']
# This Hospital's last Quarter's individual separations
EUHM_individual = Individual_Turnover[Individual_Turnover.Hospital == 'EUHM']

In [25]:
EUHM_data.head()

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
2,EUHM,U52 Renal,4,18.37,18.1,20,0,0,0.00,1.0,...,0.0,0.0,0,0,0,8.56,7.0,7,1,0
8,EUHM,EUHM Pre-op PAT,5,34.69,33.0,34,0,2,0.00,0.0,...,0.0,0.0,0,0,0,6.01,5.0,5,0,1
9,EUHM,EUHM Cath Lab,4,6.70,7.9,7,2,0,0.00,0.0,...,0.0,0.0,0,0,0,1.00,2.0,2,0,0
13,EUHM,72,5,19.13,14.5,16,0,1,1.03,1.0,...,0.0,0.0,0,0,0,7.96,9.0,10,0,0
14,EUHM,U61 Med,6,44.11,37.7,41,1,1,0.14,0.0,...,0.0,0.0,0,0,0,24.43,20.6,21,3,0


In [26]:
EUHM_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
26,EUHM,U11 Cardiac CV,4,2 years,4 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
27,EUHM,U11 Cardiac CV,4,5 years,10 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
28,EUHM,U11 Cardiac CV,4,8 years,8 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
29,EUHM,U11 Cardiac CV,4,1 year,1 year,RN,.90,"Stayed in hospital, left unit",Unknown reason
30,EUHM,U11 Cardiac CV,4,4 years,4 years,RN,.90,"Stayed in hospital, left unit",Unknown reason


## EJCH

In [27]:
NDNQI_RN_Turnover_XML('EJCH','2020 EJCH Turnover Template.xml','YTD069',2021,2)

In [28]:
NDNQI_RN_Individual_XML('EJCH','Temp_EJCH.xml',2021,2)

In [29]:
# This Hospital's Turnover Summary Data
EJCH_data = Turnover_data[Turnover_data.Hospital == 'EJCH']
# This Hospital's last Quarter's individual separations
EJCH_individual = Individual_Turnover[Individual_Turnover.Hospital == 'EJCH']

In [30]:
EJCH_data

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
26,EJCH,JC ROCU,6,7.41,9.20,8,2,0,0.0,0.0,...,0.00,0.0,0,0,0,0.59,0.0,0,0,0
33,EJCH,JC Oncology,5,15.51,14.70,15,2,0,0.0,0.0,...,0.05,0.0,0,0,0,5.33,4.5,5,0,1
42,EJCH,JC4,6,36.92,29.10,31,2,2,0.0,0.0,...,1.96,1.8,2,0,0,15.82,14.1,15,1,0
56,EJCH,JC ROCU,5,7.41,8.70,8,1,1,0.0,0.0,...,0.00,0.0,0,0,0,0.59,0.0,0,0,0
57,EJCH,JC CDU,5,8.47,7.50,7,2,0,0.0,0.0,...,0.00,0.0,0,0,0,3.29,4.6,5,0,0
76,EJCH,JC Oncology,6,15.51,13.20,14,1,2,0.0,0.0,...,0.05,0.0,0,0,0,5.33,4.5,5,0,0
89,EJCH,JC ED,6,26.26,20.40,21,2,2,0.0,0.0,...,0.79,0.0,0,0,0,7.85,6.6,6,2,1
90,EJCH,JCEndo,4,5.24,5.40,3,5,0,0.0,0.0,...,0.00,0.0,0,0,0,1.69,0.9,1,0,0
108,EJCH,JCPain,4,0.97,1.00,0,2,0,0.0,0.0,...,0.00,0.0,0,0,0,0.00,0.0,0,0,0
111,EJCH,JC ROCU,4,7.41,9.20,8,2,0,0.0,0.0,...,0.00,0.0,0,0,0,0.59,0.0,0,0,0


In [31]:
EJCH_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
85,EJCH,JC3,4,1 year,1 year,UAP,.90,Left hospital,Family obligations
86,EJCH,JC4,4,8 years,8 years,RN,.90,Left hospital,Family obligations
87,EJCH,JC Oncology,4,3 years,17 years,RN,.90,Left hospital,Performance or discipline
88,EJCH,JC ICU,4,3 years,42 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
89,EJCH,JC ICU,4,4 years,4 years,UAP,.90,Left hospital,Performance or discipline


## EDH

In [32]:
NDNQI_RN_Turnover_XML('EDH', '2021 EDH Turnover Template.xml','NMW346',2021,2)

In [33]:
NDNQI_RN_Individual_XML('EDH','Temp_EDH.xml',2021,2)

In [34]:
# This Hospital's Turnover Summary Data
EDH_data = Turnover_data[Turnover_data.Hospital == 'EDH']
# This Hospital's last Quarter's individual separations
EDH_individual = Individual_Turnover[Individual_Turnover.Hospital == 'EDH']

In [35]:
EDH_data.head()

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
12,EDH,EDH NICU,4,48.29,52.08,45,16,1,0.0,0.0,...,0.0,0.0,0,0,0,5.72,4.52,3,2,1
30,EDH,EDH 25 ICU,6,63.31,61.90,62,10,0,0.0,0.0,...,0.0,0.0,0,0,0,9.98,7.80,8,1,1
37,EDH,EDH POHA,4,12.59,10.86,4,10,0,0.0,0.0,...,0.0,0.0,0,0,0,4.57,4.72,5,0,0
46,EDH,EDH Cath Lab,5,1.00,2.00,2,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.00,0.00,0,0,0
48,EDH,EDH Labor and Delivery,6,45.21,41.16,34,17,2,0.0,0.0,...,0.0,0.0,0,0,0,10.88,9.26,7,4,0


In [36]:
EDH_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
3,EDH,EDH 25 ICU,4,Less than 1 year,20 years,RN,.60,"Stayed in hospital, left unit",Unknown reason
4,EDH,EDH 25 ICU,4,3 years,3 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
5,EDH,EDH 25 ICU,4,Less than 1 year,Less than 1 year,RN,.90,"Stayed in hospital, left unit",Unknown reason
6,EDH,EDH 25 ICU,4,3 years,14 years,RN,.60,"Stayed on unit, left permanent direct care pos...",Unknown reason
7,EDH,EDH 25 ICU,4,Less than 1 year,Less than 1 year,UAP,.90,"Stayed on unit, left permanent direct care pos...",Unknown reason


## EHH

In [37]:
NDNQI_RN_Turnover_XML('EHH','2021 EHH Turnover Template.xml','WXX116',2021,2)

In [38]:
NDNQI_RN_Individual_XML('EHH','Temp_EHH.xml',2021,2)

In [39]:
# This Hospital's Turnover Summary Data
EHH_data = Turnover_data[Turnover_data.Hospital == 'EHH']
# This Hospital's last Quarter's individual separations
EHH_individual = Individual_Turnover[Individual_Turnover.Hospital == 'EHH']

In [40]:
EHH_data.head()

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
11,EHH,EHH 5,6,32.40,35.3,37,3,3,0.0,0.0,...,0.0,0.0,0,0,0,17.48,17.1,17,3,0
20,EHH,EHH POHA,4,5.39,4.0,4,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.00,0.0,0,0,0
45,EHH,EHH Special Procedures,4,1.25,3.0,3,0,0,0.0,0.0,...,0.0,0.0,0,0,0,1.00,2.0,2,0,0
47,EHH,EHH POHA,6,5.39,4.0,4,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.00,0.0,0,0,0
53,EHH,EHH 3,4,0.00,0.0,0,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.50,8.1,9,0,0


In [41]:
EHH_individual.head()

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
153,EHH,EHH 4 ICU,4,3 years,7 years,RN,1.00,"Stayed in hospital, left unit",Unknown reason
154,EHH,EHH 4,4,2 years,2 years,RN,.90,"Stayed on unit, left permanent direct care pos...",Unknown reason
155,EHH,EHH 4,4,Less than 1 year,Less than 1 year,RN,.60,"Stayed on unit, left permanent direct care pos...",Unknown reason
156,EHH,EHH 5,4,2 years,12 years,RN,1.00,"Stayed in hospital, left unit",Unknown reason
157,EHH,EHH ED,4,3 years,6 years,RN,.60,"Stayed in hospital, left unit",Unknown reason


## ELTAC

In [42]:
NDNQI_RN_Turnover_XML('ELTAC','2021 ELTAC Turnover Template.xml','XLS464',2021,2)

In [43]:
NDNQI_RN_Individual_XML('ELTAC','Temp_ELTAC.xml',2021,2)

In [44]:
# This Hospital's Turnover Summary Data
ELTAC_data = Turnover_data[Turnover_data.Hospital == 'ELTAC']
# This Hospital's last Quarter's individual separations
ELTAC_individual = Individual_Turnover[Individual_Turnover.Hospital == 'ELTAC']

In [45]:
ELTAC_data

,Hospital,UnitName,Month,RnBudgetedFtes,RnActualFtes,RnFullTime,RnPartTime,RnNumberSeparations,AprnBudgetedFtes,AprnActualFtes,...,LpnBudgetedFtes,LpnActualFtes,LpnFullTime,LpnPartTime,LpnNumberSeparations,UapBudgetedFtes,UapActualFtes,UapFullTime,UapPartTime,UapNumberSeparations
115,ELTAC,ELTAC 4/5,4,37.87,24.3,26,1,1,0.0,0.0,...,0.0,0.0,0,0,0,40.14,28.9,30,3,0
159,ELTAC,ELTAC 4/5,6,37.87,25.8,27,2,2,0.0,0.0,...,0.0,0.0,0,0,0,40.14,27.7,28,4,1
175,ELTAC,ELTAC 4/5,5,37.87,22.2,23,2,3,0.0,0.0,...,0.0,0.0,0,0,0,40.14,28.6,29,4,1
184,ELTAC,ELTAC 5 ICU,4,12.37,6.3,7,0,0,0.0,0.0,...,0.0,0.0,0,0,0,5.00,0.0,0,0,0
286,ELTAC,ELTAC 5 ICU,5,12.37,5.4,6,0,1,0.0,0.0,...,0.0,0.0,0,0,0,5.00,0.0,0,0,0
291,ELTAC,ELTAC 5 ICU,6,12.37,0.0,0,0,6,0.0,0.0,...,0.0,0.0,0,0,0,5.00,0.0,0,0,0


In [46]:
ELTAC_individual

,Hospital,UnitName,Month,xTurnoverTenureYearsFIDUnit,xTurnoverTenureYearsFIDHospital,xTurnoverStaffCategoryFID,xTurnoverFteFID,xTurnoverSeparationTypeFID,xTurnoverPrimaryReasonFID
159,ELTAC,ELTAC 4/5,4,Less than 1 year,Less than 1 year,RN,.90,Left hospital,Join travel agency
346,ELTAC,ELTAC 5 ICU,6,2 years,2 years,RN,.90,Left hospital,Unknown reason
347,ELTAC,ELTAC 5 ICU,6,2 years,4 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
348,ELTAC,ELTAC 5 ICU,6,2 years,4 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
349,ELTAC,ELTAC 5 ICU,6,2 years,10 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
350,ELTAC,ELTAC 5 ICU,6,2 years,4 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
351,ELTAC,ELTAC 5 ICU,6,2 years,8 years,RN,.90,"Stayed in hospital, left unit",Unknown reason
352,ELTAC,ELTAC 4/5,6,2 years,2 years,RN,.90,Left hospital,Family obligations
353,ELTAC,ELTAC 4/5,6,1 year,1 year,RN,.90,Left hospital,Inability to advance
354,ELTAC,ELTAC 4/5,6,2 years,2 years,UAP,.90,"Stayed on unit, left permanent direct care pos...",Unknown reason
